In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install langchain-community langchain langgraph langchain-groq langchain-google-genai google-generativeai google-api-python-client google-auth-httplib2 google-auth-oauthlib wikipedia faiss-cpu tiktoken pypdf

In [ ]:
import os
import shutil
from google.colab import userdata
from typing import List, Literal, Dict, Any, Optional
from typing_extensions import TypedDict
# --- Core Langchain & Langgraph Imports ---
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langgraph.graph import END, StateGraph, START

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY 
print("Google API key loaded from Kaggle secrets.")

In [ ]:
# --- Configuration ---
DOCS_PATH = "/kaggle/input/poldocs"  # Create this directory and put policy .pdf files inside
FAISS_INDEX_PATH = "/kaggle/working/hr-policy-faiss-index-pdf/"
LLM_MODEL_NAME = "gemini-1.5-flash-latest" 
EMBEDDING_MODEL_NAME = "models/embedding-001" # Standard Google embedding model


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
def create_or_load_vector_store():
    """Loads PDF documents, creates embeddings, and builds/loads a FAISS index."""
    if os.path.exists(FAISS_INDEX_PATH):
        print(f"Loading existing FAISS index from: {FAISS_INDEX_PATH}")
        embeddings = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL_NAME, google_api_key=os.environ['GOOGLE_API_KEY'])
        vectorstore = FAISS.load_local(FAISS_INDEX_PATH, embeddings, allow_dangerous_deserialization=True)
        print("FAISS index loaded.")
        return vectorstore.as_retriever(search_kwargs={"k": 3})
    else:
        print(f"Creating new FAISS index. Loading PDF documents from: {DOCS_PATH}")
        # Load documents using PyPDFLoader
        # Use recursive=True if PDFs might be in subdirectories
        loader = DirectoryLoader(
            DOCS_PATH,
            glob="**/*.pdf",   # Look for PDF files
            loader_cls=PyPDFLoader, # Use the PDF loader
            show_progress=True,
            use_multithreading=True # Can speed up loading multiple PDFs
        )
        docs = loader.load()
        if not docs:
            # Changed error message to reflect PDF expectation
            raise FileNotFoundError(f"No .pdf files found in {DOCS_PATH}. Please place policy PDF documents there.")

        # Split documents (remains the same)
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=500, chunk_overlap=50)
        docs_split = text_splitter.split_documents(docs)
        # Check content after splitting (PDF loading can sometimes yield empty pages)
        docs_split = [doc for doc in docs_split if doc.page_content.strip()]
        if not docs_split:
             raise ValueError(f"PDF files in {DOCS_PATH} were loaded, but            resulted in no text content after splitting.Check PDF content/format.")
        print(f"Split {len(docs)} PDF pages/documents into {len(docs_split)}            text chunks.")

        # Create embeddings (remains the same)
        print(f"Initializing Google Embeddings: {EMBEDDING_MODEL_NAME}")
        embeddings = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL_NAME,         google_api_key=os.environ['GOOGLE_API_KEY'])

        # Create FAISS index (remains the same)
        print("Creating FAISS vector store...")
        vectorstore = FAISS.from_documents(docs_split, embeddings)

        # Save FAISS index (remains the same)
        print(f"Saving FAISS index to: {FAISS_INDEX_PATH}")
        vectorstore.save_local(FAISS_INDEX_PATH)
        print("FAISS index created and saved.")
        return vectorstore.as_retriever(search_kwargs={"k": 3})
        retriever = create_or_load_vector_store()

In [ ]:
Tool Setup (Wikipedia Only) ---
print("Setting up Wikipedia tool...")
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=2000)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
print("Wikipedia tool initialized.")

In [ ]:
LLM Setup (Using ChatGoogleGenerativeAI) ---
print(f"Initializing LLM: {LLM_MODEL_NAME}")

In [ ]:
# Let's configure it generally first. Specific tool binding happens later if needed.
llm = ChatGoogleGenerativeAI(model=LLM_MODEL_NAME,
                             temperature=0,
                             convert_system_message_to_human=True # Often helpful for Gemini
                            )
print("LLM initialized.")
Output:
LLM initialized.


Graph State Definition ---

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: The current user question.
        generation: The LLM generation.
        documents: List of documents retrieved from any source.
        chat_history: List of BaseMessages for conversation memory.
        datasource: The source chosen for the final answer ('vectorstore', 'wikipedia', 'google', 'fallback').
    """
    question: str
    generation: Optional[str] = None
    documents: Optional[List[str]] = None
    chat_history: Optional[List[BaseMessage]] = None
    datasource: Optional[Literal['vectorstore', 'wikipedia', 'google_grounded', 'fallback']] = None # Renamed 'google' source

Node Functions ---

Retrieve from Local Vector Store
def retrieve_local(state: GraphState) -> Dict[str, Any]:
    """Retrieve documents from the local FAISS vector store."""
    print("---NODE: retrieve_local---")
    question = state["question"]
    print(f"Retrieving documents for: {question}")
    # Assuming retriever was initialized globally
    retrieved_docs = retriever.invoke(question)
    doc_contents = [doc.page_content for doc in retrieved_docs]
    print(f"Retrieved {len(doc_contents)} documents locally.")
    return {"documents": doc_contents, "question": question} # Pass question along for clarity if needed

Grade Local Document Relevance (using LLM)
class GradeDocuments(BaseModel):
    """Binary score for relevance check."""
    binary_score: Literal["yes", "no"] = Field(..., description="Is the document relevant to the question, considering chat history? 'yes' or 'no'")

def grade_documents(state: GraphState) -> Dict[str, Any]:
    """Determines whether the retrieved documents are relevant to the question and history."""
    print("---NODE: grade_documents---")
    question = state["question"]
    documents = state["documents"]
    chat_history = state.get("chat_history", []) # Get history or empty list

    if not documents:
        print("No documents retrieved, grading as 'no'.")
        return {"documents": documents, "datasource": None, "grade": "no"} # Indicate no relevant source yet

    # Use LLM to grade relevance
    structured_llm_grader = llm.with_structured_output(GradeDocuments)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a grader assessing relevance of retrieved documents to a user question, considering the conversation history. Documents contain HR policies. Answer 'yes' if the documents likely contain the answer, 'no' otherwise."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "Retrieved documents:\n\n{documents}\n\nUser question: {question}")
    ])
    
    # Format documents for the prompt
    doc_string = "\n\n---\n\n".join(documents)
    
    #Create the full chain first ***
    grader_chain = prompt | structured_llm_grader
    
    print("Asking LLM to grade document relevance...")
    #Invoke the chain, not just the structured LLM ***
    response = grader_chain.invoke({
        "question": question,
        "documents": doc_string,
        "chat_history": chat_history
    })
    
    print(f"Relevance Grade: {response.binary_score}")
    if response.binary_score == "yes":
        return {"documents": documents, "datasource": "vectorstore", "grade": "yes"}
    else:
        # Documents are not relevant, clear them and decide next step
        print("Documents deemed irrelevant by grader.")
        return {"documents": None, "datasource": None, "grade": "no"} # Clear docs, no source decided yet


Route to Web Search or Fallback 
# Decision options are now 'wikipedia', 'search_generate', 'fallback'

class RouteChoice(BaseModel):
    """Route decision."""
    datasource: Literal["wikipedia", "search_generate", "fallback"] = Field(..., description="Given the user question and chat history, decide whether to search Wikipedia, perform a grounded Google search ('search_generate'), or if the question is unanswerable/absurd ('fallback').")

def route_web_or_fallback(state: GraphState) -> Dict[str, str]:
    """Routes to Wiki, Grounded Search (via Gemini), or Fallback."""
    print("---NODE: route_web_or_fallback---")
    question = state["question"]
    chat_history = state.get("chat_history", [])
    print(f"Routing question: {question}")

    # Use the main 'llm' instance for routing decision
    structured_llm_router = llm.with_structured_output(RouteChoice)

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an expert router. The user asked a question about HR policies.
        We already checked a local vector store with policies on onboarding, separation, leaves, and connect, but found nothing relevant.
        Based on the question and conversation history:
        - If it looks like a standard factual query that Wikipedia might answer (e.g., general knowledge, definitions), choose 'wikipedia'.
        - If it seems like a query requiring up-to-date information or broader context best answered by Google Search, choose 'search_generate'.
        - If the question seems nonsensical, unrelated to HR, or unanswerable even with web search, choose 'fallback'."""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}")
    ])
    
    # Create the full chain first ***
    router_chain = prompt | structured_llm_router
    
    #Invoke the chain, not just the structured LLM ***
    route_result = router_chain.invoke({
        "question": question,
        "chat_history": chat_history
    })
    
    print(f"Routing Decision: {route_result.datasource}")
    # Return key matches the conditional edge keys later
    return {"datasource_decision": route_result.datasource}

Wikipedia Search
def wiki_search(state: GraphState) -> Dict[str, Any]:
    """Perform Wikipedia search."""
    print("---NODE: wiki_search---")
    question = state["question"]
    print(f"Searching Wikipedia for: {question}")
    search_result = wiki_tool.invoke({"query": question})
    print(f"Wikipedia Result Length: {len(search_result)}")
    return {"documents": [search_result], "datasource": "wikipedia"} # Wrap in list, set source

Grounded Search and Generate
def search_and_generate(state: GraphState) -> Dict[str, Any]:
    """
    Generates an answer using the Gemini LLM, allowing it to use its
    built-in Google Search tool (grounding) if needed.
    """
    print("---NODE: search_and_generate (using Gemini Grounding)---")
    question = state["question"]
    chat_history = state.get("chat_history", [])

    # The core idea: Give Gemini the question and history, and instruct it
    # to answer, implicitly allowing it to use search if it deems necessary.
    # LangChain's ChatGoogleGenerativeAI handles the tool call automatically
    # when the model decides to use its internal search tool.

    prompt = ChatPromptTemplate.from_messages([
         ("system", """You are an HR assistant chatbot. Answer the user's question based on the conversation history and your general knowledge.
         If you need to find current information or information not available in the history, use your search capabilities.
         Provide a concise and accurate response. If you cannot find a relevant answer even after searching, state that."""),
         MessagesPlaceholder(variable_name="chat_history"),
         ("human", "{question}"),
    ])

    # Use the main llm instance. It's configured to potentially use tools.
    chain = prompt | llm

    print("Generating answer with potential grounding...")
    response = chain.invoke({
        "question": question,
        "chat_history": chat_history
        })

    generation = response.content
    print("Generation complete (with potential grounding).")
    # Indicate that Google grounding was the potential source
    return {"generation": generation, "datasource": "google_grounded"}


Generate Final Answer
def generate(state: GraphState) -> Dict[str, Any]:
    """Generates the final answer using LLM, context, and history."""
    print("---NODE: generate---")
    question = state["question"]
    documents = state["documents"]
    chat_history = state.get("chat_history", [])
    datasource = state["datasource"]

    if not documents:
        print("Generation node called with no documents. This shouldn't happen if routing is correct. Triggering fallback.")
        # This case should ideally be caught earlier, but as a safeguard:
        return fallback(state) # Call the fallback logic directly

    # Format documents and history for the prompt
    context = "\n\n---\n\n".join(documents)

    prompt = ChatPromptTemplate.from_messages([
        ("system", f"""You are an HR assistant chatbot. Answer the user's question related to Travel, Leave, annual health checks, notice period, office timings and separation policies based *only* on the provided context and conversation history.
        The context provided is from: {datasource}.
        Be concise and accurate. If the context does not contain the answer, ask Specific questions based on the contexts or explicitly state that you cannot answer based on the provided information.
        Do not make information up.
        <context>
        {{context}}
        </context>"""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ])

    chain = prompt | llm

    print("Generating final answer...")
    response = chain.invoke({
        "question": question,
        "context": context,
        "chat_history": chat_history
        })

    generation = response.content
    print("Generation complete.")
    # Return only the generation and final datasource
    return {"generation": generation, "datasource": datasource}



7. Final Generation and Absurdity Handling
Once all tools have done their job, the LLM either generates a confident response or politely communicates that no answer was found. This ensures we avoid misleading replies and maintain a high level of trust with users.

If a strong context is found, the LLM constructs a direct, well-informed response. If not, the bot shares a fallback message like:

"Sorry, I couldn’t find enough information to answer that. Could you rephrase or contact HR directly?"

This avoids hallucinations and maintains trust.



Fallback Answer
def fallback(state: GraphState) -> Dict[str, Any]:
    """Generates a fallback response if no relevant info is found or the question is absurd."""
    print("---NODE: fallback---")
    generation = "I couldn't find relevant information in our HR policies or via web search to answer your question accurately. If your query is about HR policies, please try rephrasing. Otherwise, it might be best to contact the HR department directly."
    return {"generation": generation, "datasource": "fallback"}

In [ ]:
# --- 6. Build the Graph ---
print("\nBuilding graph...")
workflow = StateGraph(GraphState)
# Add Nodes (Replace google_search with search_and_generate)
workflow.add_node("retrieve_local", retrieve_local)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("route_web_or_fallback", route_web_or_fallback)
workflow.add_node("wiki_search", wiki_search)
workflow.add_node("search_and_generate", search_and_generate)
workflow.add_node("generate", generate)
workflow.add_node("fallback", fallback)
# Define Edges

# Start -> Try local retrieval first
workflow.add_edge(START, "retrieve_local")

# After local retrieval -> Grade the documents
workflow.add_edge("retrieve_local", "grade_documents")

# Conditional edge based on grading
workflow.add_conditional_edges(
    "grade_documents",
    lambda x: x.get("grade", "no"),
    {
        "yes": "generate", # If relevant local docs, generate from them
        "no": "route_web_or_fallback", # If not relevant, decide web/fallback
    },
)

# Conditional edge for web/fallback routing (MODIFIED)
workflow.add_conditional_edges(
    "route_web_or_fallback",
    lambda x: x.get("datasource_decision"), # Check the 'datasource_decision' field
    {
        "wikipedia": "wiki_search",
        "search_generate": "search_and_generate", # Route to the new node
        "fallback": "fallback",
    }
)

# After Wikipedia search -> Generate answer from Wiki context
workflow.add_edge("wiki_search", "generate")

# After Grounded Search/Generate -> END (The node itself generates the answer)
workflow.add_edge("search_and_generate", END) # Directly to END

# Generate from local/wiki context -> END
workflow.add_edge("generate", END)

# Fallback -> END
workflow.add_edge("fallback", END)

# Compile the graph
app = workflow.compile()
print("Graph compiled successfully!")

In [ ]:
Example Invocation with History ---
print("\n--- Running Chat ---")

# Initialize chat history (outside the loop)
chat_history = []

while True:
    user_input = input("You: ")
    if user_input.lower() in ["quit", "exit", "bye"]:
        print("Chatbot: Goodbye!")
        break

    # Prepare graph input, including history
    inputs = {"question": user_input, "chat_history": chat_history}

    # Invoke the graph
    print("\nChatbot thinking...")
    final_state = app.invoke(inputs) # Get the final state after execution

    # Extract the final generation
    final_generation = final_state.get("generation", "Sorry, something went wrong.")
    print(f"Chatbot: {final_generation}")

    # Update chat history
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=final_generation))

    # Optional: Limit history size
    if len(chat_history) > 10: # Keep last 5 turns (10 messages)
        chat_history = chat_history[-10:]

    print("-" * 30) # Separator for turns